# Managing population growth

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
# mapping to the modules that make the app

sys.path.insert(0, "../..")
sys.path.insert(0, "..")
sys.path.insert(0, "../../functions/popgrowth/")

from app.connectors import cmdb_graph
from app.objects import time
from helpers import test_queries

c = cmdb_graph.CosmosdbClient()



executing local windows deployment


In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [4]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2023-08-08T15:24:35.599269+00:00 UTU:28316 >

In [5]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 28316}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

## Population Resource Consumption

In [6]:
from tools import consumption

We're going to run a resourse query test to ensure that consumption is happening. To test, update the `objid` with a planet that has consuming pops. 

In [7]:
validate_resources_updated_query = f"""
g.V().has('objid','2972908340606').out('has').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pre_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))
pre_consumption


g.V().has('objid','2972908340606').out('has').valueMap()



,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,-1,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


In [8]:

all_pops_query = f"""
g.V().has('label','pop').as('pop')
    .local(
        union(
            out('inhabits').as('location'),
            out('isOf').as('species')
            )
            .fold()).as('location','species')
        .path()
        .by(unfold().valueMap().fold())
"""

In [9]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)
pd.DataFrame(data)

,pop,location,species
0,"{'health': 0.1999999999999999, 'isIdle': 'fals...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
1,"{'health': 0.1999999999999999, 'name': 'Obattl...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
2,"{'health': 0.2499999999999999, 'name': 'Obattl...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
3,"{'health': 0.2499999999999999, 'name': 'Obattl...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
4,"{'health': 0.2499999999999999, 'name': 'Hassia...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
...,...,...,...
363,"{'health': -8.326672684688674e-17, 'name': 'Ci...","{'name': 'Taipe', 'class': 'terrestrial', 'obj...","{'name': 'Til', 'objid': '5478087080681', 'con..."
364,"{'health': -8.326672684688674e-17, 'name': 'Se...","{'name': 'Taipe', 'class': 'terrestrial', 'obj...","{'name': 'Til', 'objid': '5478087080681', 'con..."
365,"{'health': -8.326672684688674e-17, 'name': 'Ci...","{'name': 'Taipe', 'class': 'terrestrial', 'obj...","{'name': 'Til', 'objid': '5478087080681', 'con..."
366,"{'name': 'Wil Wakha', 'objid': '6451348661268'...","{'name': 'Vil', 'class': 'terrestrial', 'objid...","{'name': 'Casnaykun', 'objid': '6281354459848'..."


In [10]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df

,health,isIdle,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,username,objtype,id
0,2.000000e-01,false,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,8822366435138,0.4850,0.4305,0.505,BillmanLocal2,pop,7467985401518
1,2.000000e-01,true,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,8822366435138,0.5280,0.4400,0.459,BillmanLocal2,pop,5100638967625
2,2.500000e-01,true,Obattla Van,1800951322768,0.446,0.390,0.549,0.573,8822366435138,0.5610,0.4755,0.423,BillmanLocal2,pop,1800951322768
3,2.500000e-01,true,Obattla Rochi,0692394121482,0.452,0.395,0.514,0.546,8822366435138,0.5300,0.4625,0.354,BillmanLocal2,pop,0692394121482
4,2.500000e-01,true,Hassia Sin,0248671359952,0.623,0.215,0.704,0.569,4146242421204,0.6365,0.4258,0.662,BillmanLocal2,pop,0248671359952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,-8.326673e-17,true,Ciki Dadcasparkvoneuemar,3949013738185,0.432,0.571,0.653,0.419,0015468195520,0.5360,0.5535,0.543,notebook,pop,3949013738185
364,-8.326673e-17,true,Serzeampro Gradonderbocabanfort,0347683799439,0.513,0.351,0.465,0.353,4873232607770,0.4090,0.3800,0.612,notebook,pop,0347683799439
365,-8.326673e-17,true,Ciki Fekewakouronio,4014007243154,0.434,0.520,0.644,0.440,0015468195520,0.5420,0.5310,0.393,notebook,pop,4014007243154
366,7.000000e-01,true,Wil Wakha,6451348661268,0.467,0.714,0.526,0.538,5425248009508,0.5320,0.6230,0.426,notebook,pop,6451348661268


In [11]:
species_df = pd.DataFrame([d['species'] for d in data])
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
1,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
2,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
3,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
4,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
...,...,...,...,...,...,...,...,...,...
363,Til,5478087080681,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5478087080681
364,Til,5478087080681,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5478087080681
365,Til,5478087080681,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5478087080681
366,Casnaykun,6281354459848,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.harding@avanade.com,species,6281354459848


In [12]:
locations_df = pd.DataFrame([d['location'] for d in data])
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
1,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
2,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
3,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
4,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,Taipe,terrestrial,8490894854096,0.898,0.113,1.087,1849212388331,Lindraja,true,true,true,Billmanserver,planet,8490894854096
364,Taipe,terrestrial,8490894854096,0.898,0.113,1.087,1849212388331,Lindraja,true,true,true,Billmanserver,planet,8490894854096
365,Taipe,terrestrial,8490894854096,0.898,0.113,1.087,1849212388331,Lindraja,true,true,true,Billmanserver,planet,8490894854096
366,Vil,terrestrial,4734834255576,0.559,0.941,1.129,3344366957095,Tytal,true,true,true,william.harding@avanade.com,planet,4734834255576


First getting the list of consumption for each pop.


In [13]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)


We expand that dataset to break open the list of people in each pop. 

In [14]:
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 28316}

In [15]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

,location_id,consumes,pop,consumption,multi
0,2816844503430,Organic,84,168,True
1,2972908340606,Organic,78,156,True
2,4734834255576,Organic,9,18,True
3,5640544774717,Organic,98,196,True
4,8490894854096,Organic,99,198,True



Then we get a list of the resources available on that location. 

In [16]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2972908340606'], 'name': ['Dorf']},
   {'volume': [-1], 'objid': ['8679787925290'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['5640544774717'], 'name': ['Luraszi']},
   {'volume': [-1], 'objid': ['4182507921316'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2816844503430'], 'name': ['Penbufergui']},
   {'volume': [-1], 'objid': ['6331044458818'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['8490894854096'], 'name': ['Taipe']},
   {'volume': [-1], 'objid': ['2144566983866'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['4734834255576'], 'name': ['Vil']},
   {'volume': [1060], 'objid': ['6733191593029'], 'name': ['Organic']}]}]

In [17]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2816844503430,Organic,84,168,True,-1.0,-1.0
1,2972908340606,Organic,78,156,True,-1.0,-1.0
2,4734834255576,Organic,9,18,True,1060.0,1042.0
3,5640544774717,Organic,98,196,True,-1.0,-1.0
4,8490894854096,Organic,99,198,True,-1.0,-1.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [18]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
1    None
2    None
3    None
4    None
dtype: object

In [19]:
c.run_query(validate_resources_updated_query)
post_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))

In [20]:
pre_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,-1,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


Only the resources that are consumed will diminish, and they will stop diminishing if the level goes below `0`

In [21]:
post_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,-1,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [22]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2816844503430,Organic,84,168,True,-1.0,-1.0
1,2972908340606,Organic,78,156,True,-1.0,-1.0
2,4734834255576,Organic,9,18,True,1060.0,1042.0
3,5640544774717,Organic,98,196,True,-1.0,-1.0
4,8490894854096,Organic,99,198,True,-1.0,-1.0


In [23]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

84 pops will starve in 2816844503430
78 pops will starve in 2972908340606
cache threshold of n pops reached. Purging 31
.cache threshold of n pops reached. Purging 31
.98 pops will starve in 5640544774717
99 pops will starve in 8490894854096
cache threshold of n pops reached. Purging 31
.cache threshold of n pops reached. Purging 31
.cache threshold of n pops reached. Purging 31
.

0    None
1    None
3    None
4    None
dtype: object

Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [24]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOf').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out[0]

location_id    2816844503430
consumes             Organic
pop                       84
consumption              168
multi                   True
available               -1.0
remaining               -1.0
Name: 0, dtype: object


{'labels': [['location'], ['pop'], ['species']],
 'objects': [{'objid': ['2816844503430'], 'name': ['Penbufergui']},
  {'name': ['Tilcasport Tla'],
   'objid': ['3298484338584'],
   'health': [0.2999999999999999],
   'username': ['william.jeffrey.harding@gmail.com']},
  {'name': ['Towner'], 'objid': ['1245644824657'], 'consumes': ['[Organic]']}]}

## Looking more closely at starvation

* Grab my testing planet, with people on it
* Lower that amount of resource on that planet to 0
* run the lower health metric
* watch the people starve and die

In [25]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').valuemap()
    """)

planet = c.clean_node(c.res[0])
planet

{'name': 'Taipe',
 'class': 'terrestrial',
 'objid': '8490894854096',
 'radius': 0.898,
 'mass': 0.113,
 'orbitsDistance': 1.087,
 'orbitsId': '1849212388331',
 'orbitsName': 'Lindraja',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'username': 'Billmanserver',
 'objtype': 'planet',
 'id': '8490894854096'}

Grabbing that planets organics as well, and setting the amount of the organics to zero

In [26]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').has('name','Organic').property('volume',-1)
    """)

In [27]:
consumption_df.loc[consumption_df['location_id']==planet['objid'],['remaining','available']] = -1
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2816844503430,Organic,84,168,True,-1.0,-1.0
1,2972908340606,Organic,78,156,True,-1.0,-1.0
2,4734834255576,Organic,9,18,True,1060.0,1042.0
3,5640544774717,Organic,98,196,True,-1.0,-1.0
4,8490894854096,Organic,99,198,True,-1.0,-1.0


In [28]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').valuemap()
    """)

organics = c.clean_node(c.res[0])
organics

{'volume': -1,
 'name': 'Organic',
 'objid': '2144566983866',
 'max_volume': 1051,
 'description': 'bilogical material that can be consumed by pops',
 'replenish_rate': 10,
 'username': 'Billmanserver',
 'objtype': 'resource',
 'id': '2144566983866'}

Now that the resources are gone, the population should start to starve. 

In [29]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in 8490894854096


UnboundLocalError: local variable 'all_death' referenced before assignment

In [ ]:
health_query =f"""
    g.V().has('objid','{planet['objid']}').as('location').in('inhabits')
        .haslabel('pop').as('pop')
        .out('isOf').as('species')
        .path()
            .by(valueMap('objid','name'))
            .by(valueMap('name','objid','health','username'))
            .by(valueMap('name','objid','consumes'))
"""

In [ ]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

,name,objid,health,username
0,[Serzeampro Bilno],[9890591392190],[0.04999999999999992],[Billmanserver]
1,[Ciki Dadcas],[9068536152668],[0.04999999999999992],[Billmanserver]
2,[Ciki Y],[4377394578049],[0.04999999999999992],[Billmanserver]
3,[Serzeampro Wil],[1772981049949],[0.04999999999999992],[Billmanserver]
4,[Ciki Basma],[1247897538416],[0.04999999999999992],[Billmanserver]
...,...,...,...,...
125,[Serzeampro Wilherresserberg],[9409270335118],[0.04999999999999992],[notebook]
126,[Ciki Feehaugorskle],[5255429720397],[0.04999999999999992],[notebook]
127,[Ciki Dadcasparkvoneuemar],[3949013738185],[0.04999999999999992],[notebook]
128,[Serzeampro Gradonderbocabanfort],[0347683799439],[0.04999999999999992],[notebook]


Run the following cells again and again to watch the health go down. 

In [ ]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

130 pops will starve in 8490894854096


4    None
dtype: object

In [ ]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

,name,objid,health,username
0,[Serzeampro Bilno],[9890591392190],[-8.326672684688674e-17],[Billmanserver]
1,[Ciki Dadcas],[9068536152668],[-8.326672684688674e-17],[Billmanserver]
2,[Ciki Y],[4377394578049],[-8.326672684688674e-17],[Billmanserver]
3,[Serzeampro Wil],[1772981049949],[-8.326672684688674e-17],[Billmanserver]
4,[Ciki Basma],[1247897538416],[-8.326672684688674e-17],[Billmanserver]
...,...,...,...,...
125,[Serzeampro Wilherresserberg],[9409270335118],[-8.326672684688674e-17],[notebook]
126,[Ciki Feehaugorskle],[5255429720397],[-8.326672684688674e-17],[notebook]
127,[Ciki Dadcasparkvoneuemar],[3949013738185],[-8.326672684688674e-17],[notebook]
128,[Serzeampro Gradonderbocabanfort],[0347683799439],[-8.326672684688674e-17],[notebook]


In [ ]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

130 pops will starve in 8490894854096
cache threshold of n pops reached. Purging 31


TypeError: upload_data() missing 1 required positional argument: 'username'